<a href="https://colab.research.google.com/github/RonaldTuna/TensorIoTAssignment/blob/main/TensorIoT_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEPS:**

Download the data files from here - http://jmcauley.ucsd.edu/data/amazon/links.html

Apache spark tools locally and necessary tools

Download a review file with a million reviews

Using Jupyter notebook create a program to read the million reviews and get the following

transform date to MM-DD-YYYY format

Save the data into a table (postgres/sql server)
Save the output as a Parquet file


Upload code to Github  and complete Readme.md which anyone can understand

Send Github link to HR



Skills Learning / Tools Used:

    postgres
    workbench/J
    Jupyter notebook
    Apache spark
    Data frame transforms
    









In [1]:
# Install Java, Spark, Findspark and download a Postgresql driver
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q -nc https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

!wget -nc https://jdbc.postgresql.org/download/postgresql-42.3.5.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

File ‘postgresql-42.3.5.jar’ already there; not retrieving.



In [2]:
!wget -nc https://github.com/sbt/sbt/releases/download/v1.6.2/sbt-1.6.2.tgz
%pip install -r sbt-1.6.2.tgz
#NOT USED ATM

File ‘sbt-1.6.2.tgz’ already there; not retrieving.

ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 289, in run
    reqs = self.get_requirements(args, options, finder, session)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 386, in get_requirements
    filename, finder=finder, options=options, session=session
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_file.py", line 138, in parse_requirements
    for parsed_line in parser.parse(filename, constraint):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_file.py", line 328, in parse
    yiel

In [3]:
#install pyspark
try:
  %pyspark_version 3.x
except Exception:
  pass

!pip install pyspark

In [4]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `reviews` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS reviews;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE reviews;'

 * Starting PostgreSQL 10 database server
   ...done.
ALTER ROLE
DROP DATABASE
CREATE DATABASE


In [5]:
#setting up environment
%env REVIEWS_DATABASE_NAME=reviews
%env REVIEWS_DATABASE_HOST=localhost
%env REVIEWS_DATABASE_PORT=5432 
%env REVIEWS_DATABASE_USER=postgres
%env REVIEWS_DATABASE_PASS=postgres

In [6]:
#import pyspark and required tools
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

I extracted the reviews for Android Applications, which contains 2,638,173 reviews using wget.

I then load the reviews into a pyspark dataframe

In [7]:
#download review file from link 
!wget -nc "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android.json.gz"

#create a spark session
spark = SparkSession \
    .builder \
    .config("spark.jars", "/content/spark-3.2.1-bin-hadoop3.2/jars/postgresql-42.3.5.jar")\
    .master("local") \
    .appName("AmazonReviews") \
    .getOrCreate()
df = spark.read.json('/content/reviews_Apps_for_Android.json.gz')

File ‘reviews_Apps_for_Android.json.gz’ already there; not retrieving.



Transform data from MM DD, YYYY to MM-DD-YYYY by using to_date() to turn the string into a date object, and used date_format() to reformat to specified format.

Replace old column with new column containing reformatted dates by deleting the old column and renaming the new one.

In [8]:
df = df.withColumn('newTime', date_format(to_date('reviewTime', format='MM d, yyyy'), "MM-dd-yyyy"))
df = df.drop("reviewTime")
df = df.withColumnRenamed("newTime","reviewTime")
df.show()

+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+----------+
|      asin| helpful|overall|          reviewText|    reviewerID|        reviewerName|             summary|unixReviewTime|reviewTime|
+----------+--------+-------+--------------------+--------------+--------------------+--------------------+--------------+----------+
|B004A9SDD8|  [0, 0]|    5.0|Glad to finally s...| AUI0OLXAB3KKT|          A Customer|        Great app!!!|    1301184000|03-27-2011|
|B004A9SDD8|[12, 14]|    5.0|this app works gr...|A1ZUSQ3TC3EC4C|           A. Lissak|        Kid loves it|    1321574400|11-18-2011|
|B004A9SDD8|  [0, 0]|    4.0|We love these mon...| AC05OAXD72X1V|               Allie| Love these monkeys!|    1367366400|05-01-2013|
|B004A9SDD8|  [0, 2]|    5.0|cannot get my kin...|A2RVMFOKBVM21I|     Amazon Customer|fun fun for toddlers|    1350172800|10-14-2012|
|B004A9SDD8|  [1, 3]|    1.0|I start this app ...|A3NBSRGUWQGC

Load data into SQL table in local postgreSQL server

In [18]:
mode = "overwrite"
jdbc_url="jdbc:postgres://localhost:5433/reviews"
config = {"user":"postgres",
          "password":"postgres",  
          "driver":"com.microsoft.sqlserver.jdbc.SQLServerDriver"}


In [17]:
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

df.write \
    .format("jdbc") \
    .mode("append") \
    .option("url", "jdbc:postgres://localhost;databaseName=reviews;") \
    .option("dbtable", "review.reviews2") \
    .option("user","postgres") \
    .option("password","postgres") \
    .option("driver", "org.postgresql.Driver") \
    .save()

IllegalArgumentException: ignored

In [ ]:

df.write.jdbc(url=jdbc_url,
              table ="reviews", 
              mode=mode, properties = config)

In [20]:
df.write.jdbc(url='jdbc:postgres://localhost:5432/reviews', table='review.reviews2', properties=config)\
            .mode('append')\
            .save()

Py4JJavaError: ignored

Save dataframe as parquet file.

In [ ]:
df.write.parquet("/content/Amazon_reviews.parquet")